# 13章 e-Statを用いたふるさと納税のパネルデータ分析

In [ ]:
# 環境変数とパス設定に用いるライブラリ
import os
from dotenv import load_dotenv
# データ取得に用いるライブラリ
import time
import requests
# データ操作に用いるライブラリ
import json
import pandas as pd
# 可視化に用いるライブラリ
import plotly
import plotly.express as px
from plotly.subplots import make_subplots

In [ ]:
from estat import (
    get_metainfo,
    get_statsdata,
    cleansing_statsdata,
    colname_to_japanese,
    create_hierarchy_dataframe,
)

In [ ]:
load_dotenv()
appId = os.getenv("ESTAT_APP_ID")

## 東京都品川区はふるさと納税でどの程度税収を失ったのか

In [ ]:
# APIのエンドポイントURL
url = "https://service.api.metro.tokyo.lg.jp/api/t131091d0000000061-1fdc35f717b0500d770cc805d6402a0a-0/json?limit=5"
# リクエストヘッダー
headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}
# 空のデータ（curlの -d '{}' に相当）
data = {}
# POSTリクエストを送信
response = requests.post(url, headers=headers, data=json.dumps(data))
# ステータスコードが200（成功）かどうかを確認
if response.status_code == 200:
    # JSONデータを取得
    result = response.json()
    # 取得したデータを出力
    print(json.dumps(result, indent=4, ensure_ascii=False))
else:
    print(f"APIリクエストに失敗しました。ステータスコード: {response.status_code}")

In [ ]:
pd.json_normalize(result, record_path=["hits"]).drop(columns="row")

## 市区町村ごとの税収をe-Statで取得する

## データの前処理

In [ ]:
statsDataId = "0003172921"
meta = get_metainfo(appId, statsDataId)
metadata = meta["GET_META_INFO"]["METADATA_INF"]
total_num = metadata["TABLE_INF"]["OVERALL_TOTAL_NUMBER"]
total_num

In [ ]:
# 取得するデータのstatsDataId を指定する
statsDataId = "0003172921"

# 全データを格納するリスト
all_data = []

# 初回リクエスト
data = get_statsdata(appId, statsDataId)
value = colname_to_japanese(cleansing_statsdata(data))
all_data.append(value)

# "NEXT_KEY" が存在する限りループしてデータを取得
while "NEXT_KEY" in data["GET_STATS_DATA"]["STATISTICAL_DATA"]["RESULT_INF"]:
    # NEXT_KEY を設定
    next_key = data["GET_STATS_DATA"]["STATISTICAL_DATA"]["RESULT_INF"]["NEXT_KEY"]
    print(next_key)
    # 次のリクエスト送信
    data = get_statsdata(appId, statsDataId, params={"startPosition": next_key})
    # 次のデータを取得してリストに追加
    value = colname_to_japanese(cleansing_statsdata(data))
    all_data.append(value)
    time.sleep(2)

chihou_zaisei_df = pd.concat(all_data)

In [ ]:
chihou_zaisei_df

In [ ]:
# " 団体名( 市町村分) コード" の内容の先頭2 文字が "13" の場合に "tokyo" 列に1 を付与する
# データの接合用に、" 市区町村コード" を別の列に保存する
# さらに、" 時間軸( 年度次)" から数字部分を取り出す
chihou_zaisei_df = chihou_zaisei_df.assign(**{
    "tokyo": chihou_zaisei_df["団体名(市町村分)コード"].apply(
    lambda x: 1 if x.startswith("13") else 0),
    "市区町村コード": chihou_zaisei_df["団体名(市町村分)コード"],
    "年": chihou_zaisei_df["時間軸(年度次)"].str.extract(r"(\d+)", expand=False)
})

In [ ]:
# " 年" と " 決算収支" でグループ化し、" 値" のメディアン( 中央値) を計算
chihou_zaisei_df.groupby(["年", "決算収支"])["値"].median().reset_index()

## データの可視化

In [ ]:
revenue_a = chihou_zaisei_df.loc[
    chihou_zaisei_df["決算収支"] == "歳入総額(A)", ["年", "値"]
]
fig_a = px.box(revenue_a, x="年", y="値")
fig_a.update_layout(
    title="年ごとの歳入総額(A) の推移",
    yaxis=dict(title=" 値（千円; 対数表示）"),
)
fig_a.update_xaxes(categoryorder="category ascending")
fig_a.update_yaxes(type="log")
fig_a.update_layout(width=1000, height=500, font={"size": 18})
fig_a.show()

In [ ]:
expenditure_b = chihou_zaisei_df.loc[
    chihou_zaisei_df["決算収支"] == "歳出総額(B)", ["年", "値"]
]
fig_b = px.box(expenditure_b, x="年", y="値")
fig_b.update_layout(
    title="年ごとの歳出総額(B) の推移",
    yaxis=dict(title=" 値（千円; 対数表示）"),
)
fig_b.update_xaxes(categoryorder="category ascending")
fig_b.update_yaxes(type="log")
fig_b.update_layout(width=1000, height=500, font={"size": 18})
fig_b.show()

In [ ]:
# " 年" と " 決算収支" および "tokyo" フラグでグループ化し、" 値" のメディアンを計算
tokyo_chihou_df = chihou_zaisei_df.groupby(["年", "決算収支", "tokyo"])["値"].median().reset_index()

In [ ]:
tokyo_chihou_df

In [ ]:
# 東京都と東京都以外でデータを区分する
chihou_sainyu = chihou_zaisei_df[(chihou_zaisei_df["決算収支"] == "歳入総額(A)") & (chihou_zaisei_df["tokyo"] == 0)].copy()
tokyo_sainyu = chihou_zaisei_df[(chihou_zaisei_df["決算収支"] == "歳入総額(A)") & (chihou_zaisei_df["tokyo"] == 1)].copy()

In [ ]:
# サブプロットを作成 (縦に2つのプロット)
subplot_fig = make_subplots(rows=2, cols=1, 
    subplot_titles=(
        "東京都以外の年ごとの歳入総額(A) 分布 (Boxプロット)", 
        "東京都の年ごとの歳入総額(A) 分布 (Boxプロット)"
    )
)

# 東京都以外の箱ひげ図を作成
box_fig0 = px.box(
    chihou_sainyu[["年", "値"]].sort_values("年"), 
    x="年", 
    y="値", 
)

# 東京都の箱ひげ図を作成
box_fig1 = px.box(
    tokyo_sainyu[["年", "値"]].sort_values("年"), 
    x="年", 
    y="値", 
)

# 作成した箱ひげ図をサブプロットに追加
for trace in box_fig0.data:
    subplot_fig.add_trace(trace, row=1, col=1)

for trace in box_fig1.data:
    subplot_fig.add_trace(trace, row=2, col=1)

# レイアウトの更新
subplot_fig.update_layout(
    width=1000, height=800, font={"size": 18},
    xaxis={"ticksuffix": "年"}, xaxis2={"ticksuffix": "年"},
    showlegend=False
)
# 各サブプロットのY軸をログスケールに設定
subplot_fig.update_yaxes(type="log", row=1, col=1)
subplot_fig.update_yaxes(type="log", row=2, col=1)
# グラフを表示
subplot_fig.show()

## パネル固定効果モデルによるふるさと納税制度による効果の推定

In [ ]:
# ダミー変数の作成
chihou_zaisei_df = chihou_zaisei_df.assign(**{"furusato_dummy": (chihou_zaisei_df["年"] >= "2009").astype(int)})

In [ ]:
statsDataId = "0000020203"
meta = get_metainfo(appId, statsDataId)
metadata = meta["GET_META_INFO"]["METADATA_INF"]
total_num = metadata["TABLE_INF"]["OVERALL_TOTAL_NUMBER"]
total_num

In [ ]:
# 全データを格納するリスト
all_data = []

# 初回リクエスト
data = get_statsdata(appId, statsDataId)
value = colname_to_japanese(cleansing_statsdata(data))
all_data.append(value)

# 'NEXT_KEY' が存在する限りループしてデータを取得
while 'NEXT_KEY' in data['GET_STATS_DATA']['STATISTICAL_DATA']['RESULT_INF']:
    # NEXT_KEY を設定
    next_key = data['GET_STATS_DATA']['STATISTICAL_DATA']['RESULT_INF']['NEXT_KEY']
    print(next_key)
    
    # 次のリクエスト送信
    data = get_statsdata(appId, statsDataId, params={"startPosition": next_key})
    
    # 次のデータを取得してリストに追加
    value = colname_to_japanese(cleansing_statsdata(data))
    all_data.append(value)

taxpayer_df = pd.concat(all_data)
taxpayer_df

In [ ]:
# 納税義務者数のみを取り出す
taxpayer_df_filtered = taxpayer_df.loc[taxpayer_df["Ｃ　経済基盤"] == "C120120_納税義務者数（所得割）"].copy()

In [ ]:
taxpayer_df_filtered

In [ ]:
# 数字部分を取り出す
# " 地域" 列から市区町村名を抽出して新しい列 " 市区町村" を作成
taxpayer_df_filtered = taxpayer_df_filtered.assign(**{
    "年": taxpayer_df_filtered["調査年"].str.extract(r"(\d+)", expand=False),
    "市区町村": taxpayer_df_filtered["地域"].str.split(" ").str[1]
})

In [ ]:
# rename メソッド を用いてtaxpayer_df_filtered の " 地域コード" 列を " 市区町村コード" に変更する
taxpayer_df_filtered = taxpayer_df_filtered.rename(columns={"地域コード": "市区町村コード"})
# 年をint 型にする
chihou_zaisei_df = chihou_zaisei_df.assign(**{"年": chihou_zaisei_df["年"].astype(int)})
taxpayer_df_filtered["年"] = taxpayer_df_filtered["年"].astype(int)

In [ ]:
chihou_zaisei_df.head()

In [ ]:
#chihou_zaisei_df.to_csv("chihou_zaisei_df.csv", index=False)
#chihou_zaisei_df = pd.read_csv("chihou_zaisei_df.csv")
#taxpayer_df_filtered.to_csv("taxpayer_df_filtered.csv", index=False)
#taxpayer_df_filtered = pd.read_csv("taxpayer_df_filtered.csv", dtype={"市区町村コード": str})

In [ ]:
# " 年" と" 市区町村名" でマージ
merged_df = pd.merge(chihou_zaisei_df, taxpayer_df_filtered,on=["年", "市区町村コード"], how="left", suffixes=("", "_taxpayer"))

In [ ]:
merged_df

In [ ]:
features_df = (
    merged_df
    .assign(**{
        "year": merged_df["年"].astype(int),
        "furusatotokyo": (
            (merged_df["furusato_dummy"] == 1) & (merged_df["tokyo"] == 1)
    ).astype(int),
        "納税義務者数": merged_df["値_taxpayer"].replace("-", pd.NA)
    })
    .rename(columns={"値": "歳入総額"})
    .dropna()
    .reset_index(drop=True)
)

In [ ]:
# データを " 市区町村コード" と "year" で2 階層のマルチインデックスに設定する
features_df = features_df.set_index(["市区町村コード", "year"])

In [ ]:
features_df

In [ ]:
features_df.to_csv("features_df.csv")
#features_df = pd.read_csv("features_df.csv")

In [ ]:
# インデックスのレベル（階層）名
print(f" インデックスのレベル名: {features_df.index.names}")

# 第1レベル "市区町村コード" のユニークな数
num_entities_from_index = features_df.index.get_level_values("市区町村コード").nunique()
print(f"市区町村数: {num_entities_from_index}")

# 第2レベル "year" のユニークな数
num_time_from_index = features_df.index.get_level_values("year").nunique()
print(f"カバー年数: {num_time_from_index}")

In [ ]:
!pip install linearmodels

In [ ]:
import statsmodels.api as sm
from linearmodels.panel import PanelOLS

# 説明変数リストを作成する
# ここでは、説明変数として「ふるさと納税ダミー」と「ふるさと納税x 東京都ダミー」、
# それから " 納税義務者数" を用いる
exog_vars = ["furusato_dummy", "furusatotokyo", "納税義務者数"]

# 説明変数に定数項を追加する
exog_data = features_df[exog_vars]
exog_data = sm.add_constant(exog_data)

# 被説明変数は歳入総額を用いる
endog_data = features_df["歳入総額"]

# exog_data と endog_data のデータ長を確認する
print("Length of exog_data:", len(exog_data))
print("Length of endog_data:", len(endog_data))

# exog_data とendog_data のインデックスが一致していることを確認する
print("Index of exog_data:", exog_data.index.names)
print("Index of endog_data:", endog_data.index.names)

# exog_data とendog_data のデータ確認する
print("exog_data の列名:", exog_data.columns)
print("exog_data の形状:", exog_data.shape)
print("endog_data の形状:", endog_data.shape)

# fixed effect（市区町村の固定効果）で分析する
mod = PanelOLS(endog_data, exog_data, entity_effects=True)

# モデルの結果を表示する
fe_res = mod.fit()
print(fe_res)

In [ ]:
formula_fe2 = "歳入総額 ~ furusatotokyo + 納税義務者数 + 1 + TimeEffects + EntityEffects"
result_fe2 = PanelOLS.from_formula(formula_fe2, data=features_df).fit()
print(result_fe2)

# 【ここからは本に載せていない部分】

## 変動効果モデル

In [ ]:
from linearmodels.panel import RandomEffects

# 説明変数リストを作成する
# ここでは、説明変数として「ふるさと納税ダミー」と「ふるさと納税x 東京都ダミー」、
# それから " 納税義務者数" を用いる
exog_vars = ["furusato_dummy", "furusatotokyo", "納税義務者数"]

# 説明変数に定数項を追加する
exog_data = features_df[exog_vars]
exog_data = sm.add_constant(exog_data)

# 被説明変数は歳入総額を用いる
endog_data = features_df["歳入総額"]

# exog_data と endog_data のデータ長を確認する
print("Length of exog_data:", len(exog_data))
print("Length of endog_data:", len(endog_data))

# exog_data とendog_data のインデックスが一致していることを確認する
print("Index of exog_data:", exog_data.index.names)
print("Index of endog_data:", endog_data.index.names)

# exog_data とendog_data のデータ確認する
print("exog_data の列名:", exog_data.columns)
print("exog_data の形状:", exog_data.shape)
print("endog_data の形状:", endog_data.shape)

# fixed effect（市区町村の固定効果）で分析する
mod2 = RandomEffects(endog_data, exog_data)

# モデルの結果を表示する
re_res = mod2.fit()
print(re_res)

## モデルの比較

In [ ]:
from linearmodels.panel import compare
print(compare({'FE':fe_res,'FE2': result_fe2, 'RE':re_res}))

## ハウスマン検定

In [ ]:
import numpy as np
from scipy import stats

# ハウスマン検定（Hausman Test）の実行 ---
# FEモデルとREモデルのどちらが適切かを統計的に判断する
#
# 帰無仮説 (H0): REモデルが適切（個体効果と説明変数は相関しない）
# 対立仮説 (H1): FEモデルが適切（個体効果と説明変数は相関している）

def hausman_test(fe_results, re_results):
    """
    固定効果モデルとランダム効果モデルのハウスマン検定を計算する

    Parameters:
    - fe_results: 固定効果モデルの推定結果 (linearmodels)
    - re_results: ランダム効果モデルの推定結果 (linearmodels)

    Returns:
    - chi2: カイ二乗統計量
    - df: 自由度
    - p_value: p値
    """
    # FEとREで共通して推定されるパラメータを取得
    # FEでは時間不変な変数は除外されるため
    common_params = list(set(fe_results.params.index) & set(re_results.params.index))

    # 共通パラメータの係数と共分散行列を抽出
    b_fe = fe_results.params[common_params]
    b_re = re_results.params[common_params]
    cov_fe = fe_results.cov.loc[common_params, common_params]
    cov_re = re_results.cov.loc[common_params, common_params]

    # 係数の差と共分散行列の差を計算
    b_diff = b_fe - b_re
    cov_diff = cov_fe - cov_re

    # ハウスマン検定統計量を計算
    # H = (b_fe - b_re)' * inv(Cov(b_fe) - Cov(b_re)) * (b_fe - b_re)
    try:
        inv_cov_diff = np.linalg.inv(cov_diff)
    except np.linalg.LinAlgError:
        # 行列が特異（singular）で逆行列を計算できない場合
        print("Warning: Covariance matrix difference is singular. Hausman test cannot be performed.")
        return np.nan, np.nan, np.nan

    chi2_stat = b_diff.T @ inv_cov_diff @ b_diff

    # p値を計算
    df = len(common_params)
    p_value = 1 - stats.chi2.cdf(chi2_stat, df)

    return chi2_stat, df, p_value

# 検定を実行し、結果を表示
chi2, df, p_value = hausman_test(fe_res, re_res)

print("\n### Hausman Test for FE vs. RE ###")
print(f"Chi-squared statistic: {chi2:.4f}")
print(f"Degrees of freedom: {df}")
print(f"P-value: {p_value:.4f}")

# --- 検定結果を解釈する ---
print("\n--- Interpretation of Hausman Test ---")
if p_value < 0.05:
    print("P-value is less than 0.05. We reject the null hypothesis.")
    print("Conclusion: The individual-specific effects are likely correlated with the regressors.")
    print("Therefore, the Fixed-Effects (FE) model is the more appropriate choice for this analysis.")
else:
    print("P-value is not less than 0.05. We fail to reject the null hypothesis.")
    print("Conclusion: There is no statistical evidence that individual-specific effects are correlated with the regressors.")
    print("Therefore, the more efficient Random-Effects (RE) model is preferred.")

## 被説明変数を歳入総額の変化率にして計算する

In [ ]:
# 「値」の前年の値をシフトして取得
features_df['lagged_歳入総額'] = features_df.groupby(level=0)['歳入総額'].shift(1)
# 値の昨年との変化率を計算
features_df['change_rate'] = (features_df['歳入総額'] - features_df['lagged_歳入総額'])/features_df['lagged_歳入総額']

# inf と -inf を NaN に置換する
features_df.replace([np.inf, -np.inf], np.nan, inplace=True)

# 'change_rate'列がNaNである行（infから変換されたものも含む）を削除する
features_df2 = features_df.dropna(subset=['change_rate'])

formula_fe3 = 'change_rate ~ furusatotokyo + 納税義務者数 + 1 +  EntityEffects'
result_fe3 = PanelOLS.from_formula(formula_fe3, data=features_df2).fit()
print(result_fe3)

In [ ]:
formula_fe4 = 'change_rate ~ furusatotokyo + 納税義務者数 + 1 + TimeEffects + EntityEffects'
result_fe4 = PanelOLS.from_formula(formula_fe4, data=features_df2).fit()
print(result_fe4)